In [3]:
import os 
import sys
sys.path.append(os.path.join('../..'))
# sys.path.append(os.path.join(os.environ['HAWQ_JET_TAGGING'], 'utilities'))

from pulp import *
import numpy as np
import torch

In [2]:
from utilities.compute_bops import compute_bops
from model import AutoEncoder

2023-11-13 04:55:18.820315: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-13 04:55:18.897846: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-13 04:55:19.731844: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/jcampos/miniforge3/lib/python3.9/site-packages/ot/backend.py:2998: UserWarning: To use TensorflowBackend, you need to activate the tensorflow numpy API. You can activate it by running: 
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
  register_backend(TensorflowBackend())


#### 1. Setup

In [19]:
checkpoint_file = "/data/jcampos/hawq-jet-tagging/checkpoints/econ/10.31.2023-18.50.41/last.ckpt"

model = AutoEncoder(
    accelerator="auto", 
    quantize=False,
    precision=[32, 32, 32],
    learning_rate=1e-3,  
    econ_type="baseline",
)

model.load_state_dict(torch.load(checkpoint_file)["state_dict"])

<All keys matched successfully>

In [21]:
import torchinfo
torchinfo.summary(model, input_size=(1, 1, 8, 8))  # (B, C, H, W)

/home/jcampos/miniforge3/lib/python3.9/site-packages/torchinfo/torchinfo.py:462: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),


Layer (type:depth-idx)                   Output Shape              Param #
AutoEncoder                              [1, 1, 8, 8]              --
├─BaseEncoder: 1-1                       [1, 16]                   --
│    └─Conv2d: 2-1                       [1, 8, 4, 4]              80
│    └─ReLU: 2-2                         [1, 8, 4, 4]              --
│    └─Flatten: 2-3                      [1, 128]                  --
│    └─Linear: 2-4                       [1, 16]                   2,064
│    └─ReLU: 2-5                         [1, 16]                   --
├─Sequential: 1-2                        [1, 1, 8, 8]              --
│    └─Linear: 2-6                       [1, 128]                  2,176
│    └─ReLU: 2-7                         [1, 128]                  --
│    └─Unflatten: 2-8                    [1, 8, 4, 4]              --
│    └─ConvTranspose2d: 2-9              [1, 8, 8, 8]              584
│    └─ReLU: 2-10                        [1, 8, 8, 8]              --
│    └─C

1.1 Quantization error

In [14]:
MIN_BITWIDTH = 2
MAX_BITWIDTH = 8
LAYERS = ['conv', 'enc_dense']

all_bit_widths = list(range(MIN_BITWIDTH,MAX_BITWIDTH+1))
delta_weights = {}  # store the L2 norm. 

for bit_width in all_bit_widths:
    tmp_delta_weights = []
    
    for idx, layer_name in enumerate(LAYERS):
        # get layer to compute quant. error 
        layer = getattr(model.encoder, layer_name)
        
        # min and max for chosen bitwidth 
        q_min = -(2**bit_width)
        q_max = (2**bit_width)-1

        # quantize and dequantize weights
        x = torch.clamp(layer.weight, q_min, q_max)
        delta = (q_max-q_min)/(q_max-1)
        x_integer = torch.round((x-q_min)/delta)
        x = x_integer*delta+q_min

        # compute the L2 norm. 
        l2_weight_perturbation = ((x.reshape(1,-1) - layer.weight.reshape(1,-1))**2).sum()
        l2_weight_perturbation = l2_weight_perturbation.detach().numpy().item()
        
        # store resut
        tmp_delta_weights.append(l2_weight_perturbation)

    delta_weights[bit_width] = np.array(tmp_delta_weights)


for bit_width in all_bit_widths:
    print(f"{bit_width} {delta_weights[bit_width]}")
    # print(f"{bit_width} {delta_weights[bit_width]/np.array([ 1088, 2080, 1056, 165]).sum()}")

2 [ 23.66148376 986.60961914]
3 [ 23.52730751 718.11633301]
4 [ 22.68955231 632.29559326]
5 [ 22.25978661 597.82232666]
6 [ 22.08265686 582.24053955]
7 [ 22.0026474  574.79370117]
8 [ 21.96077538 571.20532227]


In [15]:
# convert to numpy array 
l2_quant_pert = []

for bit_width in delta_weights.keys():
    l2_quant_pert.append(delta_weights[bit_width])

l2_quant_pert = np.array(l2_quant_pert)
print(l2_quant_pert)

[[ 23.66148376 986.60961914]
 [ 23.52730751 718.11633301]
 [ 22.68955231 632.29559326]
 [ 22.25978661 597.82232666]
 [ 22.08265686 582.24053955]
 [ 22.0026474  574.79370117]
 [ 21.96077538 571.20532227]]


1.2 BOPs

#### 2.0 - Integer Linear Programming 

In [16]:
class args():
    def __init__(self):
        pass

args = args
args.model_size_limit = 0.5 
args.bops_limit = 0.0003 
args.latency_limit = 0.5 

In [8]:
# Number of layers (for ILP setup)
NUM_LAYERS = len(LAYERS)

# number of paramers of each layer
parameters = np.array([ 80, 2064])

# Hutchinson_trace means the trace of Hessian for each weight matrix.
Hutchinson_trace = np.array([5.3138685, 0.8988645])  # original (not normalized)

# BOPs of each layer
bops_2bit = np.array([55296, 47104, 47104, 3520])
bops_3bit = np.array([72704, 67584, 32768, 5120])
bops_4bit = np.array([90112, 92160, 45056, 7040]) 
bops_5bit = np.array([107520, 120832, 59392, 9280])
bops_6bit = np.array([124928, 153600, 75776, 11840])
bops_7bit = np.array([142336, 190464, 94208, 14720])
bops_8bit = np.array([159744, 231424, 114688, 17920])
bops_9bit = np.array([340992, 215040, 106496, 16640])
bops_10bit = np.array([374784, 258048, 128000, 20000])
bops_32bit = np.array([1118208, 2240512, 831119232, 174880])

# bops = np.array([bops_2bit, bops_3bit, bops_4bit, bops_5bit, bops_6bit, bops_7bit, bops_8bit, bops_9bit, bops_10bit]).reshape(-1) / 1024
bops = np.array([bops_4bit, bops_5bit, bops_6bit, bops_7bit, bops_8bit]).reshape(-1) 

In [9]:
# model size 
model_size_32bit = np.sum(parameters) 
# model_size_limit = model_size_32bit * args.model_size_limit

# bops
# BOPS_LIMIT = np.sum(bops_32bit) * args.bops_limit 
BOPS_LIMIT = int(350e3)

### 2.1 - ILP BOPs Size Constrait

In [10]:
# construct the problem 
number_variables = Hutchinson_trace.shape[0]*len(l2_quant_pert) # NUM_LAYERS * BIT_WIDTH_OPTIONS

# first get the variables
variable = {}
for i in range(number_variables):
    variable[f"x{i}"] = LpVariable(f"x{i}", 0, 1, cat=LpInteger)


In [11]:
prob = LpProblem("Model_Size", LpMinimize)

# add objective function, minimize model size 
# prob += sum([variable[f"x{i}"] * parameters[i%4] for i in range(num_variable) ]) <= model_size_limit 

# add objective function, minimize bops 
prob += sum([variable[f"x{i}"] * bops[i] for i in range(number_variables) ]) <= BOPS_LIMIT 

In [12]:
# Each layer has BIT_WIDTH_OPTIONS variables (5 in this case), each variable can either be 0 or 1 
# an extra constraint is needed to chose one bitwidth per layer
prob += sum([variable[f"x{i}"] for i in list(range(0, number_variables, NUM_LAYERS))]) == 1  
prob += sum([variable[f"x{i}"] for i in list(range(1, number_variables, NUM_LAYERS))]) == 1
prob += sum([variable[f"x{i}"] for i in list(range(2, number_variables, NUM_LAYERS))]) == 1
prob += sum([variable[f"x{i}"] for i in list(range(3, number_variables, NUM_LAYERS))]) == 1

In [13]:
prob += sum( [ variable[f"x{i}"] * l2_quant_pert.reshape(-1)[i] * Hutchinson_trace[i%NUM_LAYERS] for i in range(number_variables) ] ) 

In [14]:
# solve the problem
status = prob.solve(GLPK_CMD(msg=1, options=["--tmlim", "10000","--simplex"]))
# get the result
LpStatus[status]

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/b5caeafe96a04a58a6190ae648347b7e-pulp.lp -o /tmp/b5caeafe96a04a58a6190ae648347b7e-pulp.sol
 --tmlim 10000 --simplex
Reading problem data from '/tmp/b5caeafe96a04a58a6190ae648347b7e-pulp.lp'...
5 rows, 20 columns, 40 non-zeros
20 integer variables, all of which are binary
39 lines were read
GLPK Integer Optimizer 5.0
5 rows, 20 columns, 40 non-zeros
20 integer variables, all of which are binary
Preprocessing...
5 rows, 20 columns, 40 non-zeros
20 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.314e+05  ratio =  2.314e+05
GM: min|aij| =  7.612e-01  max|aij| =  1.314e+00  ratio =  1.726e+00
EQ: min|aij| =  5.805e-01  max|aij| =  1.000e+00  ratio =  1.723e+00
2N: min|aij| =  3.438e-01  max|aij| =  1.000e+00  ratio =  2.909e+00
Constructing initial basis...
Size of triangular part is 5
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
5 rows, 20 columns, 

'Optimal'

In [15]:
# reshape ILP result 
result = []
for i in range(number_variables):
    result.append(value(variable[f"x{i}"]))

result

result = np.array(result).reshape(len(l2_quant_pert),-1)
bitwidth_idxs = np.argmax(result, axis=0)

In [16]:
print(f"Bit Width\tDense1\t\tDense2\t\tDense3\t\tDense4")
for idx in range(len(l2_quant_pert)):
    print(f"{all_bit_widths[idx]:2} \
          {result[idx][0]:4d} \
            {result[idx][1]:3d} \
            {result[idx][2]:3d} \
            {result[idx][3]:3d}")

# get total bops
total_bops = 0
for idx in range(NUM_LAYERS):
    bops_index = bitwidth_idxs[idx]*NUM_LAYERS + idx
    total_bops += bops[bops_index]

print(f"Total BOPs: {total_bops}")
print(f"BOPs limit: {BOPS_LIMIT}")

Bit Width	Dense1		Dense2		Dense3		Dense4
 4              0               0               0               0
 5              0               1               1               1
 6              0               0               0               0
 7              0               0               0               0
 8              1               0               0               0
Total BOPs: 349248
BOPs limit: 350000
